In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
import pandas as pd
import numpy as np
import seaborn as sns

In [99]:
df = pd.read_csv('/Users/kasper/OneDrive/study/UvA/Master/Data mining/Assignments/Assignment 1 - advanced/dataset_mood_smartphone.csv')
df.head()

,Unnamed: 0,id,time,variable,value
0,1,AS14.01,2014-02-26 13:00:00.000,mood,6.0
1,2,AS14.01,2014-02-26 15:00:00.000,mood,6.0
2,3,AS14.01,2014-02-26 18:00:00.000,mood,6.0
3,4,AS14.01,2014-02-26 21:00:00.000,mood,7.0
4,5,AS14.01,2014-02-27 09:00:00.000,mood,6.0


In [100]:
pd.melt(df, id_vars=['variable'], var_name="variable", value_name="value")

/Users/kasper/opt/anaconda3/envs/dataMining_assignment1_py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  exec(code_obj, self.user_global_ns, self.user_ns)


,variable,variable,value
0,Unnamed: 0,Unnamed: 0,1
1,Unnamed: 0,Unnamed: 0,2
2,Unnamed: 0,Unnamed: 0,3
3,Unnamed: 0,Unnamed: 0,4
4,Unnamed: 0,Unnamed: 0,5
...,...,...,...
1507643,value,value,8.032
1507644,value,value,3.008
1507645,value,value,7.026
1507646,value,value,23.033


In [78]:
df["value"].isna().sum() #sum the total NaN values in the "value" column

202

In [79]:
df["value"] = df["value"].fillna(0) #fill NaN values with 0

In [72]:
display(df.dtypes) #check data types

Unnamed: 0      int64
id             object
time           object
variable       object
value         float64
dtype: object

In [73]:
df["value"] = df["value"].astype(int)

In [80]:
display(df.dtypes) #check again to see if value column changed to integer
df.head()

Unnamed: 0      int64
id             object
time           object
variable       object
value         float64
dtype: object

,Unnamed: 0,id,time,variable,value
0,1,AS14.01,2014-02-26 13:00:00.000,mood,6.0
1,2,AS14.01,2014-02-26 15:00:00.000,mood,6.0
2,3,AS14.01,2014-02-26 18:00:00.000,mood,6.0
3,4,AS14.01,2014-02-26 21:00:00.000,mood,7.0
4,5,AS14.01,2014-02-27 09:00:00.000,mood,6.0


In [81]:
df = df["value"].to_numpy()
mean = df[:800].mean(axis=0)
df -= mean
std = df[:800].std(axis=0)
df /= std

In [82]:
def generator(data, lookback, delay, start, end, batch_size = 64):
    if end is None:
        end = len(data) - delay - 1
    i = start + lookback
    while True:
        if i + batch_size >= end:
            i = start + lookback
        rows = np.arange(i, min(i + batch_size, end))
        i += len(rows)
        samples = np.zeros((len(rows),
                           lookback,
                           data.shape[-1]))
        res_s = np.zeros((len(rows),))

        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j])
            samples[j] = data[indices]
            tar_v = data[rows[j] + delay][3]
            if tar_v > 0:
                res_s[j] = 1
            else:
                res_s[j] = 0
        yield samples, res_s

In [83]:
lookback = 10
delay = 1
batch_size = 128
steps_per_epc = int(800/batch_size)

In [93]:
train_generator = generator(df,
                           lookback = lookback,
                           delay = delay,
                           start = 0,
                           end = 800,
                           batch_size = batch_size)
val_generator = generator(df,
                           lookback = lookback,
                           delay = delay,
                           start = 801,
                           end = 1100,
                           batch_size = batch_size)
test_generator = generator(df,
                           lookback = lookback,
                           delay = delay,
                           start = 1101,
                           end = None,
                           batch_size = batch_size)

In [94]:
val_steps = (1100 - 801 - lookback)
test_steps = (len(df) - 1101 - lookback)

In [95]:
model_rnn = Sequential()
model_rnn.add(layers.Flatten(input_shape = (lookback, df.shape[-1])))
model_rnn.add(layers.Dense(32,activation = 'relu'))
model_rnn.add(layers.Dropout(0.3))
model_rnn.add(layers.Dense(1,activation = 'sigmoid'))
model_rnn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 3769120)           0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                120611872 
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 33        
Total params: 120,611,905
Trainable params: 120,611,905
Non-trainable params: 0
_________________________________________________________________


In [96]:
model_rnn.compile(optimizer = RMSprop(lr = 1e-2), loss = 'binary_crossentropy', metrics = ['acc'])
history = model_rnn.fit_generator(train_generator, steps_per_epoch=steps_per_epc, epochs = 20, validation_data = val_generator, validation_steps = val_steps)

Epoch 1/20


ValueError: could not broadcast input array from shape (10) into shape (10,376912)

In [88]:
acc_ = history_dic['loss']
val_acc_ = history_dic['val_loss']
epochs = range(1,21)
#plt.clf()
plt.plot(epochs,acc_, 'bo', label = "training loss")
plt.plot(epochs, val_acc_, 'r', label = "validation loss")
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

NameError: name 'history_dic' is not defined

In [20]:
scores = model_rnn.evaluate_generator(test_generator,test_steps) 
print("Accuracy = ", scores[1]," Loss = ", scores[0])

KeyError: "None of [Int64Index([1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110], dtype='int64')] are in the [columns]"